In [ ]:
import pandas as pdi
import datetime as dt
from scipy.stats import itemfreq

import numpy as np

In [ ]:
from sklearn import metrics
from sklearn.cluster import KMeans,MiniBatchKMeans

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
data= pd.read_excel("data.xlsx",sep='|')

In [ ]:
data.head(3)

In [ ]:
data=data[data['Objective'].notnull()]


In [ ]:
data=data[data.Status!='Inactive']

In [ ]:
data=data.dropna(axis=0, how='all')

In [ ]:
data['Objective'].shape

In [ ]:
#data['Objective1']=data['Objective'].apply(lambda x : x.replace('_',' '))

In [ ]:
#print(data['Name1'])

In [ ]:
data['Objective'].shape

In [ ]:
data.shape

In [ ]:
NullValuesInDescription =sum(pd.isnull(data['Objective']))
print(NullValuesInDescription)

In [ ]:
import nltk
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [ ]:
#list=['affects', 'affined','affinity','affinity_notchanged', 'aggregation','aggressive', 'alarm','alarms','alignment', 'all_ssr', 'allocation','android', 'anti','antutu','api','apk','app','application', 'apps', 'apt']

In [ ]:
"""
new_text = 'app apps alarm alrms affined affinity'
words = nltk.word_tokenize(new_text)
final = [[ps.stem(token) for token in sentence.split(" ")] for sentence in list]
for w in words:
    print(ps.stem(w))"""

#### remove punctuations !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

In [ ]:
import string
table = str.maketrans(' ', ' ', string.punctuation)
#table= str.maketrans(string.punctuation, ' '*len(string.punctuation))
data['Obj'] = [w.translate(table) for w in data['Objective']]
    

In [ ]:
print(string.punctuation)

In [ ]:
data['Obj'] =[' '.join(s for s in words.split() if not (any(c.isdigit() for c in s) or (len(s)<3))) for words in data['Obj'] ]

In [ ]:
doc=data['Obj'].values

In [ ]:
len(doc)

In [ ]:
#data['Obj']

In [ ]:
doc

In [ ]:
#writer=pd.ExcelWriter('data1.xlsx')
#data.to_excel(writer,'Sheet1')
#writer.save()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
#from stemming.porter2 import stem

token_dict = {}
stemmer = PorterStemmer()


def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
        #stemmed.append(stem(item))    
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

        
tfidf = TfidfVectorizer(norm='l1',analyzer='word',tokenizer = tokenize,stop_words='english')
tfs = tfidf.fit_transform(doc)

In [ ]:
len(tfidf.get_feature_names())

In [ ]:
print(tfs[:1])

In [ ]:
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

In [ ]:
#print(tfidf.get_feature_names())

In [ ]:
#tfidf.get_feature_names()

In [ ]:
len(tfidf.vocabulary_)

In [ ]:
print(tfs[0])

In [ ]:
doc[:3]

In [ ]:
data = tfs.toarray()

In [ ]:
reduced_data = PCA(n_components=100).fit_transform(data)

In [ ]:
r_Data=TruncatedSVD(n_components=100).fit_transform(tfs)

In [ ]:
#print(tfs)

In [ ]:
import sklearn
from sklearn.cluster import SpectralClustering
model = SpectralClustering(n_clusters=10, affinity = sklearn.metrics.pairwise.cosine_similarity)

In [ ]:
from nltk.cluster.kmeans import KMeansClusterer
kclusterer = KMeansClusterer(3, distance=sklearn.metrics.pairwise.cosine_similarity, repeats=50)
#clusters = kclusterer.cluster(r_Data,True, trace=True)

In [ ]:
true_k = 10
model = KMeans(n_clusters=10, init='k-means++', max_iter=1000,algorithm='full',precompute_distances=True)
model.fit(reduced_data)

In [ ]:
model.labels_[209:231] 

In [ ]:
itemfreq(model.labels_)

In [ ]:
#{i: np.where(model.labels_ == i)[0] for i in range(model.n_clusters)}

### MiniBatchKmeans

In [ ]:
km = MiniBatchKMeans(n_clusters = 10, init='k-means++', n_init=1, batch_size=100, verbose=1)

In [ ]:
km.fit(r_Data)
labels = km.labels_

In [ ]:
itemfreq(labels)

## Top words in cluster

In [ ]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = tfidf.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i,)
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind],)
    print
from __future__ import print_function
print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = model.cluster_centers_.argsort()[:, ::-1] 
for i in range(true_k):
print("Cluster %d words:" % i, end='')
for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
print() #add whitespace
print() #add whitespace
print("Cluster %d titles:" % i, end='')
for title in frame.ix[i]['title'].values.tolist():
print(' %s,' % title, end='')
print() #add whitespace
print() #add whitespace
print()
print()

## Agglomerative clustering

In [ ]:
from math import sqrt


def dot_product(v1, v2):
    """Get the dot product of the two vectors.
    if A = [a1, a2, a3] && B = [b1, b2, b3]; then
    dot_product(A, B) == (a1 * b1) + (a2 * b2) + (a3 * b3)
    true
    Input vectors must be the same length.
    """
    return sum(a * b for a, b in zip(v1, v2))


def magnitude(vector):
    """Returns the numerical length / magnitude of the vector."""
    return sqrt(dot_product(vector, vector))


def similarity(v1, v2):
    """Ratio of the dot product & the product of the magnitudes of vectors."""
    return dot_product(v1, v2) / (magnitude(v1) * magnitude(v2) + .00000000001)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering


In [ ]:
len(reduced_data[0])

In [ ]:
C = 1 - cosine_similarity(reduced_data)
ward = AgglomerativeClustering(n_clusters=10,linkage='complete',affinity='cosine').fit(r_Data)
label = ward.labels_

In [ ]:
itemfreq(label)

### DBSCAN ALgorithm 

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
db = DBSCAN(eps=0.19, min_samples=10).fit(r_Data)

In [ ]:
labels =db.labels_

In [ ]:
np.unique(labels)

In [ ]:
itemfreq(labels)

In [ ]:
from sklearn.cluster import FeatureAgglomeration

In [ ]:
len(r_Data[0])

In [ ]:
#data = FeatureAgglomeration(affinity='cosine',n_clusters=10,linkage='complete').fit_transform(data)

In [ ]:
db=FeatureAgglomeration(affinity='cosine',n_clusters=10,linkage='average').fit(r_Data)

In [ ]:
db.labels_

In [ ]:
np.unique(labels)

In [ ]:
itemfreq(labels)

In [ ]:
import matplotlib.pyplot as plt
reduced_data = PCA(n_components=2).fit_transform(data)
kmeans = KMeans(init='k-means++', n_clusters=10, n_init=10)
kmeans.fit(reduced_data)

# Step size of the mesh. Decrease to increase the quality of the VQ.
h = .02     # point in the mesh [x_min, x_max]x[y_min, y_max].

# Plot the decision boundary. For that, we will assign a color to each
x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Obtain labels for each point in mesh. Use last trained model.
Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1)
plt.clf()
plt.imshow(Z, interpolation='nearest',
           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
           cmap=plt.cm.Paired,
           aspect='auto', origin='lower')

plt.plot(reduced_data[:, 0], reduced_data[:, 1], 'k.', markersize=2)
# Plot the centroids as a white X
centroids = kmeans.cluster_centers_
plt.scatter(centroids[:, 0], centroids[:, 1],
            marker='x', s=169, linewidths=3,
            color='w', zorder=10)
plt.title('K-means clustering (PCA-reduced data)\n'
          'Centroids are marked with white cross')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())
plt.show()

In [ ]:
labels=kmeans.labels_

In [ ]:
itemfreq(labels)